![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow%2FTensorFlow&dt=TensorFlow+Basics+-+Data+To+Training.ipynb)

# TensorFlow Basics - Data To Training

How to retrieve data for training models with TensorFlow.

This workflow covers getting data to the location of training, in this case, many methods of getting BigQuery data into a Pandas DataFrame.  Then the dataframe is used as inputs for batches to TensorFlow with named inputs (columns).  Additionally, the TensorFlow I/O reader for BigQuery is used to directly read batches from BigQuery without the need to first load an entire dataframe.

**TensorFlow Basics**

This workflow is part of a [series](./readme.md) focused as getting started guides for users with familiarity with other frameworks and wanting to get started using TensorFlow.

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20Autoencoders/Autoencoders%20-%20Data.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.bigquery_storage', 'google-cloud-bigquery-storage'),
    ('bigframes', 'bigframes'),
    ('pandas_gbq', 'pandas-gbq'),
    ('tensorflow', 'tensorflow', '2.10'),
    ('tensorflow_io', '--no-deps tensorflow-io'),
    ('graphviz', 'graphviz'),
    ('pydot', 'pydot')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [4]:
#!sudo apt-get -qq install graphviz

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
EXPERIMENT = 'data'
SERIES = 'tensorflow'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id,splits' # add more variables to the string with comma delimiters

packages:

In [92]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import bigquery
from google.cloud import bigquery_storage
import bigframes.pandas as bpd
import pandas as pd
import numpy as np
import concurrent.futures

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

#from datetime import datetime

#from google.protobuf import json_format
#from google.protobuf.struct_pb2 import Value
#import json



clients:

In [9]:
bq = bigquery.Client(project = PROJECT_ID)
bqstorage = bigquery_storage.BigQueryReadClient()
bpd.options.bigquery.project = PROJECT_ID

---
## Review Data

The data source here was prepared in [01 - BigQuery - Table Data Source](../../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb).  In this notebook we will use prepared BigQuery table as input for TensorFlow.

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction

The data preparation included added splits for machine learning with a column named `splits` with 80% for training (`TRAIN`), 10% for validation (`VALIDATE`) and 10% for testing (`TEST`).  Additionally, a unique identifier was added to each transaction, `transaction_id`.  

Review the number of records for each level of Class (VAR_TARGET) for each of the data splits:

In [10]:
query = f"""
    SELECT splits, {VAR_TARGET}, count(*) as n
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    GROUP BY splits, {VAR_TARGET}
"""
print(query)


    SELECT splits, Class, count(*) as n
    FROM `statmike-mlops-349915.fraud.fraud_prepped`
    GROUP BY splits, Class



In [11]:
bq.query(query = query).to_dataframe()

,splits,Class,n
0,TEST,0,28455
1,TEST,1,47
2,TRAIN,0,227664
3,TRAIN,1,397
4,VALIDATE,0,28196
5,VALIDATE,1,48


Further review the balance of the target variable (VAR_TARGET) for each split as a percentage of the split:

In [12]:
query = f"""
    WITH
        COUNTS as (SELECT splits, {VAR_TARGET}, count(*) as n FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` GROUP BY splits, {VAR_TARGET})

    SELECT *,
        SUM(n) OVER() as total,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY {VAR_TARGET})) as n_pct_class,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY splits)) as n_pct_split,
        SAFE_DIVIDE(SUM(n) OVER(PARTITION BY {VAR_TARGET}), SUM(n) OVER()) as class_pct_total
    FROM COUNTS
"""
print(query)


    WITH
        COUNTS as (SELECT splits, Class, count(*) as n FROM `statmike-mlops-349915.fraud.fraud_prepped` GROUP BY splits, Class)

    SELECT *,
        SUM(n) OVER() as total,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY Class)) as n_pct_class,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY splits)) as n_pct_split,
        SAFE_DIVIDE(SUM(n) OVER(PARTITION BY Class), SUM(n) OVER()) as class_pct_total
    FROM COUNTS



In [13]:
review = bq.query(query = query).to_dataframe()
review

,splits,Class,n,total,n_pct_class,n_pct_split,class_pct_total
0,TEST,1,47,284807,0.095528,0.001649,0.001727
1,TEST,0,28455,284807,0.100083,0.998351,0.998273
2,TRAIN,1,397,284807,0.806911,0.001741,0.001727
3,TRAIN,0,227664,284807,0.800746,0.998259,0.998273
4,VALIDATE,1,48,284807,0.097561,0.001699,0.001727
5,VALIDATE,0,28196,284807,0.099172,0.998301,0.998273


---
## From BigQuery To Pandas DataFrame

### Common Query

In [14]:
query = f'''
SELECT * EXCEPT({','.join(VAR_OMIT.replace(' ', '').split(','))})
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
WHERE splits = 'TRAIN'
'''
print(query)


SELECT * EXCEPT(transaction_id,splits)
FROM `statmike-mlops-349915.fraud.fraud_prepped`
WHERE splits = 'TRAIN'



### BigQuery Cell Magic

https://cloud.google.com/python/docs/reference/bigquery/latest/magics

In [15]:
%%bigquery bq_data_magic
SELECT * EXCEPT(transaction_id,splits)
FROM `statmike-mlops-349915.fraud.fraud_prepped`
WHERE splits = 'TRAIN'

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
bq_data_magic.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,2812,-0.633403,0.963616,2.494946,2.099051,-0.404331,0.235862,-0.007932,0.211442,-0.209817,...,0.014676,0.016278,-0.061462,0.355196,-0.179086,-0.106947,-0.215039,0.050698,0.0,0
1,3150,1.313281,-0.257923,0.118463,-0.735557,-0.569308,-0.733577,-0.138659,-0.141641,1.708019,...,-0.082467,0.126066,-0.223157,-0.074977,0.921940,-0.528283,0.064476,0.013132,0.0,0
2,16676,1.158480,0.168947,0.536345,1.187908,-0.265547,-0.076325,-0.355844,0.144615,1.462346,...,0.016492,0.263518,-0.076711,-0.079402,0.502827,-0.270819,-0.004966,-0.003372,0.0,0
3,17701,-1.279231,-0.153303,3.296310,3.320441,1.139018,0.542343,-0.729928,-0.051774,0.922712,...,-0.409746,-0.342575,-0.493297,-0.017046,-0.107404,0.101164,-0.197940,-0.435654,0.0,0
4,28131,1.069507,-0.000362,1.448936,2.874498,-0.736266,0.831932,-0.762267,0.406772,0.626473,...,0.035393,0.444433,-0.085413,0.099090,0.506438,0.246418,0.057864,0.021133,0.0,0


In [17]:
type(bq_data_magic)

pandas.core.frame.DataFrame

In [18]:
bq_data_magic.shape

(228061, 31)

### BigQuery Python Client

https://cloud.google.com/python/docs/reference/bigquery/latest

In [19]:
bq_data_client = bq.query(query = query).to_dataframe()
bq_data_client.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,2812,-0.633403,0.963616,2.494946,2.099051,-0.404331,0.235862,-0.007932,0.211442,-0.209817,...,0.014676,0.016278,-0.061462,0.355196,-0.179086,-0.106947,-0.215039,0.050698,0.0,0
1,3150,1.313281,-0.257923,0.118463,-0.735557,-0.569308,-0.733577,-0.138659,-0.141641,1.708019,...,-0.082467,0.126066,-0.223157,-0.074977,0.921940,-0.528283,0.064476,0.013132,0.0,0
2,16676,1.158480,0.168947,0.536345,1.187908,-0.265547,-0.076325,-0.355844,0.144615,1.462346,...,0.016492,0.263518,-0.076711,-0.079402,0.502827,-0.270819,-0.004966,-0.003372,0.0,0
3,17701,-1.279231,-0.153303,3.296310,3.320441,1.139018,0.542343,-0.729928,-0.051774,0.922712,...,-0.409746,-0.342575,-0.493297,-0.017046,-0.107404,0.101164,-0.197940,-0.435654,0.0,0
4,28131,1.069507,-0.000362,1.448936,2.874498,-0.736266,0.831932,-0.762267,0.406772,0.626473,...,0.035393,0.444433,-0.085413,0.099090,0.506438,0.246418,0.057864,0.021133,0.0,0


In [20]:
type(bq_data_client)

pandas.core.frame.DataFrame

In [21]:
bq_data_client.shape

(228061, 31)

### BigQuery BigFrames Client

https://cloud.google.com/python/docs/reference/bigframes/latest

In [22]:
bq_data_bigframes = bpd.read_gbq(query)
bq_data_bigframes.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,72890,-1.22258,-0.017622,2.317581,-1.547722,-0.958068,-0.370571,-0.583838,0.384328,-0.72238,...,0.430025,1.217131,-0.463494,0.456253,0.385304,-0.104713,-0.303068,-0.300302,5.9,0
1,131206,1.967597,-1.009301,-1.970656,-0.406056,1.614598,3.92548,-1.209586,0.952736,-0.429297,...,-0.288566,-0.420307,0.258054,0.632264,-0.148758,-0.656398,0.077885,-0.027551,59.0,0
2,122831,2.290614,-1.288035,-1.091499,-1.591945,-0.983697,-0.58711,-0.952236,-0.272064,-1.392405,...,-0.161871,0.04143,0.225622,0.672485,-0.105101,-0.194599,0.007488,-0.040725,30.0,0
3,68397,1.258859,0.440981,0.331167,0.681581,-0.267935,-1.046229,0.163925,-0.269223,-0.142249,...,-0.27486,-0.734847,0.116306,0.376938,0.25547,0.090629,-0.015355,0.033149,0.89,0
4,152137,2.023988,-0.351874,-0.494781,0.36047,-0.400929,-0.202362,-0.544039,-0.078031,1.364484,...,0.160192,0.774027,0.021697,-0.601828,0.029147,-0.175735,0.04743,-0.041086,9.99,0


In [23]:
type(bq_data_bigframes)

bigframes.dataframe.DataFrame

In [24]:
bq_data_bigframes.shape

(228061, 31)

In [25]:
bq_data_bigframes = bq_data_bigframes.to_pandas()
type(bq_data_bigframes)

pandas.core.frame.DataFrame

In [26]:
bq_data_bigframes.shape

(228061, 31)

### BigQuery Storage Client

https://cloud.google.com/python/docs/reference/bigquerystorage/latest

In [27]:
read_session = bqstorage.create_read_session(
    request = dict(
        parent = f'projects/{PROJECT_ID}',
        read_session = dict(
            table = f"projects/{BQ_PROJECT}/datasets/{BQ_DATASET}/tables/{BQ_TABLE}",
            data_format = bigquery_storage.types.DataFormat.ARROW,
            read_options = dict(
                row_restriction = "splits = 'TRAIN'",
                selected_fields = bq_data_bigframes.columns.tolist()
            )
        ),
        max_stream_count = 0
    )
)

In [28]:
len(read_session.streams)

1

In [29]:
def read_stream(stream):
    # setup a reader
    reader = bqstorage.read_rows(name = stream.name)
    # read rows from reader into a dataframe.  Note this is actually multiple operations - read and convert
    return reader.to_dataframe()


bq_data_storage = []
with concurrent.futures.ThreadPoolExecutor(max_workers = len(read_session.streams)) as executor:
    futures = {
        executor.submit(read_stream, stream): stream for stream in read_session.streams
    }
    for future in concurrent.futures.as_completed(futures):
        stream = futures[future]
        bq_data_storage.append(future.result())

In [30]:
len(bq_data_storage)

1

In [31]:
bq_data_storage[0].shape

(228061, 31)

In [32]:
bq_data_storage = pd.concat(bq_data_storage)
bq_data_storage.shape

(228061, 31)

In [33]:
type(bq_data_storage)

pandas.core.frame.DataFrame

In [34]:
bq_data_storage.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,2812,-0.633403,0.963616,2.494946,2.099051,-0.404331,0.235862,-0.007932,0.211442,-0.209817,...,0.014676,0.016278,-0.061462,0.355196,-0.179086,-0.106947,-0.215039,0.050698,0.0,0
1,3150,1.313281,-0.257923,0.118463,-0.735557,-0.569308,-0.733577,-0.138659,-0.141641,1.708019,...,-0.082467,0.126066,-0.223157,-0.074977,0.921940,-0.528283,0.064476,0.013132,0.0,0
2,16676,1.158480,0.168947,0.536345,1.187908,-0.265547,-0.076325,-0.355844,0.144615,1.462346,...,0.016492,0.263518,-0.076711,-0.079402,0.502827,-0.270819,-0.004966,-0.003372,0.0,0
3,17701,-1.279231,-0.153303,3.296310,3.320441,1.139018,0.542343,-0.729928,-0.051774,0.922712,...,-0.409746,-0.342575,-0.493297,-0.017046,-0.107404,0.101164,-0.197940,-0.435654,0.0,0
4,28131,1.069507,-0.000362,1.448936,2.874498,-0.736266,0.831932,-0.762267,0.406772,0.626473,...,0.035393,0.444433,-0.085413,0.099090,0.506438,0.246418,0.057864,0.021133,0.0,0


### Indirect BigQuery with `pandas-gbq`

When working with [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) the methods above show the client returning data to pandas dataframes.  This section will show a pandas mudule, [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/) that wraps the BigQuery client so that pandas can retrieve BigQuery data to dataframes.

References:
- [Comparison of BigQuery Client with pandas-gbq](https://cloud.google.com/bigquery/docs/pandas-gbq-migration)

In [35]:
bq_data_pandasgbq = pd.read_gbq(query, project_id = PROJECT_ID)
bq_data_pandasgbq.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,2812,-0.633403,0.963616,2.494946,2.099051,-0.404331,0.235862,-0.007932,0.211442,-0.209817,...,0.014676,0.016278,-0.061462,0.355196,-0.179086,-0.106947,-0.215039,0.050698,0.0,0
1,3150,1.313281,-0.257923,0.118463,-0.735557,-0.569308,-0.733577,-0.138659,-0.141641,1.708019,...,-0.082467,0.126066,-0.223157,-0.074977,0.921940,-0.528283,0.064476,0.013132,0.0,0
2,16676,1.158480,0.168947,0.536345,1.187908,-0.265547,-0.076325,-0.355844,0.144615,1.462346,...,0.016492,0.263518,-0.076711,-0.079402,0.502827,-0.270819,-0.004966,-0.003372,0.0,0
3,17701,-1.279231,-0.153303,3.296310,3.320441,1.139018,0.542343,-0.729928,-0.051774,0.922712,...,-0.409746,-0.342575,-0.493297,-0.017046,-0.107404,0.101164,-0.197940,-0.435654,0.0,0
4,28131,1.069507,-0.000362,1.448936,2.874498,-0.736266,0.831932,-0.762267,0.406772,0.626473,...,0.035393,0.444433,-0.085413,0.099090,0.506438,0.246418,0.057864,0.021133,0.0,0


In [36]:
type(bq_data_pandasgbq)

pandas.core.frame.DataFrame

In [37]:
bq_data_pandasgbq.shape

(228061, 31)

## From Pandas Dataframe To TensorFlow

The methods above read data to a Pandas dataframe that is local to this session.  This section shows how to make the dataframe ready for TensorFlow as a `tf.data` object.  More methods are [covered here](https://www.tensorflow.org/tutorials/load_data/pandas_dataframe) in the TensorFlow tutorials.

### Features as a Feature Array:

The following constructs a reader that converts the feature columns into a feature array.  This can be helpful when all the columns have the same data type, however, it does require providing data in the same column order at serving time which could create difficulty.  For name inputs see the section that follows this one.

In [53]:
training_data = bq_data_storage.copy()

In [54]:
nclasses = training_data[VAR_TARGET].unique().shape[0]

In [55]:
def prep(features):
    target = features.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float64)
    return(features, target)

In [56]:
training_reader_array = tf.data.Dataset.from_tensor_slices(prep(training_data))

In [57]:
for features, target in training_reader_array.batch(5).take(1):
    print('features:\n', features)
    print('\ntarget:\n', target)

features:
 tf.Tensor(
[[ 2.81200000e+03 -6.33402988e-01  9.63616039e-01  2.49494562e+00
   2.09905099e+00 -4.04330673e-01  2.35861580e-01 -7.93190515e-03
   2.11441518e-01 -2.09816820e-01  3.08297603e-01 -1.20499231e+00
  -4.74707809e-01 -6.54063562e-01 -4.74599113e-01 -4.28417793e-01
   5.36651482e-01 -3.80654617e-01  2.86505393e-02 -6.87969434e-01
  -1.74984760e-01  1.46755278e-02  1.62781766e-02 -6.14624729e-02
   3.55196343e-01 -1.79085504e-01 -1.06947425e-01 -2.15039257e-01
   5.06977952e-02  0.00000000e+00]
 [ 3.15000000e+03  1.31328087e+00 -2.57922822e-01  1.18462828e-01
  -7.35556647e-01 -5.69307720e-01 -7.33577207e-01 -1.38659177e-01
  -1.41641341e-01  1.70801916e+00 -1.10329377e+00 -1.08782009e+00
   6.44675882e-01 -2.15368642e-01 -7.47149675e-02  2.87873335e-01
  -1.00176397e+00  9.37677555e-02 -7.25490596e-02  1.10838530e+00
  -1.45143829e-01 -8.24673673e-02  1.26065912e-01 -2.23157253e-01
  -7.49771434e-02  9.21939582e-01 -5.28283485e-01  6.44756571e-02
   1.31318875e-02  

### Features as named inputs:

This approach maps each column to an input with the column name.  Basically, a dictionary of columns.  The additional step is casting the dataframe as a dictionary with `dict(df)`.  When columns have different data types or when they need to be named inputs to help with serving the model later.

In [59]:
training_data = bq_data_storage.copy()

In [60]:
nclasses = training_data[VAR_TARGET].unique().shape[0]

In [61]:
def prep(features):
    target = features.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float64)
    return(features, target)

In [62]:
training_reader = tf.data.Dataset.from_tensor_slices(dict(training_data))

In [64]:
for features, target in training_reader.map(prep).batch(5).take(1):
    print('features:\n', list(features.keys()), '\n')
    for feature in features.items():
        print(feature)
    print('\ntarget:\n', target)

features:
 ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'] 

('Time', <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 2812,  3150, 16676, 17701, 28131])>)
('V1', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.63340299,  1.31328087,  1.15847976, -1.27923083,  1.06950736])>)
('V2', <tf.Tensor: shape=(5,), dtype=float64, numpy=
array([ 9.63616039e-01, -2.57922822e-01,  1.68946502e-01, -1.53303332e-01,
       -3.61567697e-04])>)
('V3', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([2.49494562, 0.11846283, 0.53634505, 3.29631037, 1.44893558])>)
('V4', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 2.09905099, -0.73555665,  1.18790841,  3.32044136,  2.87449797])>)
('V5', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.40433067, -0.56930772, -0.26554689,  1.13901754, -0.73626591])>)
('V6', <tf.Tensor: shape=(

### Training In TensorFlow

Using the features as named inputs here:

In [65]:
feature_inputs = [tf.keras.Input(shape = (1,), dtype = dtypes.float64, name = feature) for feature in training_data.columns if feature != VAR_TARGET]

Define the models layers:

In [66]:
feature_layer = tf.keras.layers.Concatenate(name = 'feature_layer')(feature_inputs)
norm_layer = tf.keras.layers.BatchNormalization(axis = -1, name = 'batch_normalization')(feature_layer)
logistic = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'logistic')(norm_layer)

Group the layers into a model:

In [67]:
model = tf.keras.Model(
    inputs = feature_inputs,
    outputs = logistic,
    name = 'example_from_dataframe'
)

Compile the model for training:

In [68]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(), #SGD or Adam
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'PR', name = 'auprc')]
)

Train the model:

In [69]:
model.fit(training_reader.prefetch(2).map(prep).shuffle(1000).batch(100), epochs = 2)

Epoch 1/2
2281/2281 [==============================] - 13s 5ms/step - loss: 0.0698 - accuracy: 0.9836 - auprc: 0.9982
Epoch 2/2
2281/2281 [==============================] - 12s 5ms/step - loss: 0.0093 - accuracy: 0.9989 - auprc: 0.9996


Use the model for prediction:

In [91]:
prediction = model.predict(training_reader_tfio.batch(1).map(prep).take(1))
prediction

1/1 [==============================] - 0s 380ms/step


array([[0.996509  , 0.00349101]], dtype=float32)

Interpret the prediction to get the predicted classification level:

In [95]:
np.argmax(prediction)

0

## From BigQuery To TensorFlow With TensorFlow I/O

A highly effective way to read batches directly to `tf.data` objects from BigQuery storage!

https://www.tensorflow.org/io

In [70]:
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses = nclasses.shape[0]
nclasses

2

In [71]:
query = f'''
SELECT *
FROM {BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = '{BQ_TABLE}'
    AND COLUMN_NAME NOT IN ('transaction_id', 'splits')
'''
schema = bq.query(query).to_dataframe()

In [72]:
schema.data_type.unique().tolist()

['INT64', 'FLOAT64']

In [73]:
types = {
    'FLOAT64' : dtypes.float64,
    'INT64' : dtypes.int64
}

In [74]:
def prep(features):
    target = features.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float64)
    return(features, target)

In [75]:
training_reader_tfio = BigQueryClient().read_session(
    parent = f"projects/{PROJECT_ID}",
    project_id = BQ_PROJECT,
    table_id = BQ_TABLE,
    dataset_id = BQ_DATASET,
    selected_fields = [x for x in schema.column_name.tolist()],
    output_types = [types[x] for x in schema.data_type.tolist()],
    row_restriction = f"splits='TRAIN'",
    requested_streams = 3
).parallel_read_rows(sloppy = True, num_parallel_calls = tf.data.experimental.AUTOTUNE)
type(training_reader_tfio)

tensorflow.python.data.ops.interleave_op._ParallelInterleaveDataset

In [76]:
for features, target in training_reader_tfio.map(prep).batch(5).take(1):
    print('features:\n', list(features.keys()), '\n')
    for feature in features.items():
        print(feature)
    print('\ntarget:\n', target)

features:
 ['Amount', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9'] 

('Amount', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0., 0., 0., 0., 0.])>)
('Time', <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 2812,  3150, 16676, 17701, 28131])>)
('V1', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.63340299,  1.31328087,  1.15847976, -1.27923083,  1.06950736])>)
('V10', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 0.3082976 , -1.10329377, -0.17276001,  0.84594969,  0.37324618])>)
('V11', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-1.20499231, -1.08782009,  2.05305928,  1.38923569, -1.32944263])>)
('V12', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.47470781,  0.64467588, -2.73649895, -2.44018135, -0.1867695 ])>)
('V13', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.65406356, -0.21536864, -

### Training In TensorFlow

Using the features as named inputs here:

In [77]:
feature_inputs = [tf.keras.Input(shape = (1,), dtype = dtypes.float64, name = feature) for feature in schema.column_name if feature != VAR_TARGET]

Define the models layers:

In [78]:
feature_layer = tf.keras.layers.Concatenate(name = 'feature_layer')(feature_inputs)
norm_layer = tf.keras.layers.BatchNormalization(axis = -1, name = 'batch_normalization')(feature_layer)
logistic = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'logistic')(norm_layer)

Group the layers into a model:

In [79]:
model = tf.keras.Model(
    inputs = feature_inputs,
    outputs = logistic,
    name = 'example_from_dataframe'
)

Compile the model for training:

In [80]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(), #SGD or Adam
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'PR', name = 'auprc')]
)

Train the model:

In [81]:
model.fit(training_reader_tfio.prefetch(2).map(prep).shuffle(1000).batch(100), epochs = 2)

Epoch 1/2
2281/2281 [==============================] - 16s 7ms/step - loss: 0.0643 - accuracy: 0.9827 - auprc: 0.9958
Epoch 2/2
2281/2281 [==============================] - 15s 7ms/step - loss: 0.0101 - accuracy: 0.9985 - auprc: 0.9995


Use the model for prediction:

In [91]:
prediction = model.predict(training_reader_tfio.batch(1).map(prep).take(1))
prediction

1/1 [==============================] - 0s 380ms/step


array([[0.996509  , 0.00349101]], dtype=float32)

Interpret the prediction to get the predicted classification level:

In [94]:
# get index of predicted classification level:
np.argmax(prediction)

0

## Understanding the Model

The models trained above takes named input features and outputs predictions for classification.  The target variable had 2 levels and the prediction includes probability of membership in either of these and the largest probability would be the predicted class.

This section revists the model setup and training and expands the explanation of each step.